In [4]:
# 指标openrank对应的数据表格式如下：
#  1.     项目1：      "2015-01": 2.07,
#                     "2015-02": 2.78,
#                     "2015-03": 3.93,
#                     "2015-04": 3.72,
#                         ……
#  2.     项目2：      "2015-01": 3.28,
#         ……              ……
# 即项目名：年-月-openrank当前值
#
# 希望能够将每个月的数据单独化，并且将时间与openrank当前值分为两列，呈现：
#  1.     项目1：      "2015-01"           2.07
#  2.     项目1：      "2015-02"           2.78
#  3.     项目1：      "2015-03"           3.93
#  4.     项目1：      "2015-04"           3.72
#                      ……
#  n.     项目2：      "2015-01"           3.28
#         ……           ……
# 这种形式方便对于每个项目的每月openrank值作绘制

import pymysql
import json

# 连接到 MySQL 数据库
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='cptbtptp789',
    database='project_data',
    charset='utf8mb4'
)

cursor = conn.cursor()

# 删除原表（如果存在）
drop_table_query = "DROP TABLE IF EXISTS openrank_metrics_rebuilt"
try:
    cursor.execute(drop_table_query)
    print("原表 openrank_metrics_rebuilt 已删除（如果存在）")
except pymysql.Error as e:
    print(f"删除原表时出错: {e}")

# 创建新表
create_table_query = """
CREATE TABLE IF NOT EXISTS openrank_metrics_rebuilt (
    id INT AUTO_INCREMENT PRIMARY KEY,
    project_name VARCHAR(255) NOT NULL,
    month VARCHAR(7) NOT NULL,
    value DECIMAL(10, 2)
)
"""
try:
    cursor.execute(create_table_query)
    print("新表 openrank_metrics_rebuilt 创建成功")
except pymysql.Error as e:
    print(f"创建新表时出错: {e}")

# 从原表查询数据
select_query = "SELECT id, project_name, metric_value FROM openrank_metrics"
cursor.execute(select_query)

# 获取查询结果
results = cursor.fetchall()

# 处理并插入数据到新表
for row in results:
    id_value, project_name, metric_value_str = row
    metric_value = json.loads(metric_value_str)
    for month_key, value in metric_value.items():
        if isinstance(value, (int, float)):
            month = month_key[:7]
            insert_query = "INSERT INTO openrank_metrics_rebuilt (project_name, month, value) VALUES (%s, %s, %s)"
            try:
                cursor.execute(insert_query, (project_name, month, value))
                conn.commit()
            except pymysql.Error as e:
                print(f"插入数据时出错: {e}")

# 关闭连接
cursor.close()
conn.close()

原表 openrank_metrics_rebuilt 已删除（如果存在）
新表 openrank_metrics_rebuilt 创建成功
